<div class="alert alert-info">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Привет. Меня зовут Алина Гусева, я буду делать ревью к твоей работе по исследовательскому анализу данных. В дальнейшем предлагаю общаться на "ты".
    
    
Видно, что ты старалась, делая проект. Все ключевые этапы проекта выполнены, в целом тебе удалось со всем справиться. Особенно хочется отметить, что соблюдена структура проекта, а код написан аккуратно.
    
Для того, чтобы и тебе и мне удобнее бы
    ло ориентироваться в комментариях, я буду помечать их разными цветами:
- красный цвет означает комментарии, которые требуют доработки. Как только ты их исправишь - проект будет принят;
- желтый цвет - то, что в следующий раз можно будет сделать по-другому. На твое усмотрение можешь доделать маленькие правки сейчас или учесть замечание для выполнения следующих проектов;
- зеленый цвет - очень элегантные и удачные решение, которые желательно использовать в дальнейшей работе.
- синий цвет - информация
    
Предлагаю работать над проектом в диалоге: если ты что-то изменяешь или вносишь какие-то коррективы - выделяй это каким-нибудь цветом и пиши об этом мне. Так мне гораздо легче будет отследить сделанные изменения.
    
Также попрошу тебя не изменять и не удалять мои комментарии, чтобы проверка твоего самостоятельного проекта прошла оперативнее.
    
 
</div>

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
print(df.head(10))
df.info()
df.columns

print(df['family_status'].unique())
print(df['family_status_id'].unique())
print(df['children'].unique())
print(df['dob_years'].unique())

print(len(df[(df['days_employed'] < 300000)]['days_employed']))
print(len(df[(df['days_employed'] < 0)]['days_employed']))
print(len(df[(df['days_employed'] >= 300000)]['days_employed']))

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

**Вывод**

Каждая строка таблицы содержит информацию о заёмщике. Для проверки рабочих гипотез особенно ценны столбцы children, family_status, total_income, purpose, debt. 

Проблемы: 

пропуски в столбцах days_employed и total_income, 

артефакты в столбце days_employed (например, 340266 / 365 = 932 года) и отрицательное количество проработанных дней,


артефакты в столбце children. Предположительно, -1 ребенок и 20 детей - это 1 и 2 ребенка,

артефакты в столбце dob_years (0 лет)

дубликаты в столбцах education (разный регистр букв) и purpose (нужна лемматизация).

Предположительно, отрицательные значения в столбце days_employed соответствуют реальному трудовому стажу и могут быть заменены опусканием минуса, а остальные положительные значения все нереалистичны и равны 300000+ дней, заменим их нулями и будем считать, что эти данные неизвестны. 

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Молодец, хороший тон использовать метод info() для получения общей информации из таблицы. Для более детального изучения можно воспользоваться методом describe(). 
    
 
</div>

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
df.isnull().sum()
print(df['income_type'].value_counts())


NaNs_before_removing = df[df['total_income'].isna()]['income_type'].value_counts()
print(NaNs_before_removing)
for income_type in df['income_type'].unique():
    total_income_median = df[df['income_type'] == income_type] ['total_income'].median()
    df.loc[df['income_type'] == income_type, 'total_income'] = df[df['income_type'] == income_type]['total_income'].fillna(total_income_median)
    days_employed_median = df[df['income_type'] == income_type]['days_employed'].median()
    df.loc[df['income_type'] == income_type, 'days_employed'] = df[df['income_type'] == income_type]['days_employed'].fillna(days_employed_median)
df.isnull().sum()

print(df[df['dob_years'] == 0 ]['income_type'].value_counts())
dob_years_mean_retiree = df[(df['dob_years'] != 0) & (df['income_type'] == 'пенсионер')]['dob_years'].mean()
dob_years_mean_others = df[(df['dob_years'] != 0) & (df['income_type'] != 'пенсионер')]['dob_years'].mean()

print(dob_years_mean_retiree)
print(dob_years_mean_others)

df.loc[(df['dob_years'] == 0) & (df['income_type'] != 'пенсионер'), 'dob_years'] = dob_years_mean_others
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'пенсионер'), 'dob_years'] = dob_years_mean_retiree

df.loc[df['days_employed'] >= 300000, 'days_employed'] = 0
df.loc[df['days_employed'] < 0, 'days_employed'] = - df.loc[df['days_employed'] < 0, 'days_employed']
print(len(df[(df['days_employed'] >= 300000)]['days_employed']))
print(df['days_employed'].head(10))
df.info()

df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)
print(df['children'].unique())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64
сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64
сотрудник      55
пенсионер      20
компаньон      20
госслужащий     6
Name: income_type, dtype: int64
59.370959332638165
40.03542187855356
0
0    8437.673028
1    4024.803754
2    5623.422610
3    4124.747207
4       0.000000
5     926.185831
6    2879.202052
7     152.779569
8    6929.865299
9    2188.756445
Name: days_employed, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-n

**Вывод**

Пропуска в total_income и days_employed идут вместе.
Возможные причины пропусков: данные были не полностью перенесены в общую таблицу. Например, в предыдущей таблице не было этих пунктов (total_income и days_employed). Пропущено около 10% значений по доходу и трудовому стажу, причем пропуски равномерно распределены по основным категориям (сотрудник, компаньон, пенсионер, госслужащий), в каждой примерно по 10%

Пропуски заменены по каждой категории (сотрудник, компаньон, пенсионер, госслужащий) на медианные, чтобы избежать влияния редких или артефактных значений.

Нулевые значения возраста заменены на средние среди пенсионеров для пенсионеров и на средние среди всех остальных для всех остальных категорий

-1 и 20 детей заменены на 1 и 2

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
1. Стаж и зарплата. Да, ты верно заметила, что пропуски в двух столбцах симметричны.  Заменять медианой некорректно. Ты выбрала самый оптимальный способ, предварительно сгруппировав данные по типу занятости. Кстати, в данном случае очень удобно воспользоваться функцией <a href='https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transform.html'>transform()</a>.  Данные в колонке ос стоажем некорректные. Об этом необходимо сообщить разработчикам базы данных во избежании дальнейших неприятностей.
    
    
2. Количество детей. Все верно, артефакты -1 и 20 необходимо заменить на более реальные значения.
</div>

### Замена типа данных

In [3]:
df.info()
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df['dob_years'] = df['dob_years'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_stat

**Вывод**

После обработки пропусков изменился тип 'dob_years', поменяем его обратно. Типы столбцов 'days_employed' и 'total_income' тоже переведем в int.

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Отлично, теперь все столбцы имеют корректный тип данных.
</div>

### Обработка дубликатов

**Вывод**

In [4]:
df.info()
df['education'] = df['education'].str.lower()
print(df.head(10))
print(df.duplicated().sum())
df = df.drop_duplicates().reset_index(drop=True) 
print(df.duplicated().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   среднее             1   
3         3           4124         32   среднее             1   
4         0        

Обнаружена и удалена 71 строка явных дубликатов.

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Здесь важно обратить внимание на то, что сначала необходимо привести текстовый формат данных к единому виду(регистр), а затем проверять на дубликаты. Иначе результат может получится другой.
 
</div>

### Лемматизация

In [5]:
df.info()
print(df['purpose'].unique())
from pymystem3 import Mystem
m = Mystem() 

lemmas = []

for i in range(len(df['purpose'])):
    lemmas += m.lemmatize(df.loc[i,'purpose'])


purpose_id = [];
purpose_type = [];
queries = ['недвижимость', 'автомобиль', 'образование', 'свадьба', 'жилье']

for purpose in df['purpose']:
    for i in range(len(queries)):
        if queries[i] in m.lemmatize(purpose):
            if (queries[i] == 'недвижимость') or (queries[i] == 'жилье'):
                purpose_type.append('недвижимость')
                purpose_id.append(0)
            else:
                purpose_type.append(queries[i])
                purpose_id.append(i)
                        
df['purpose_type'] = purpose_type
df['purpose_id'] = purpose_id
print(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвиж

**Вывод**

Можно выделить следующие преобладающие категории в столбце с целями: недвижимость, жилье, автомобиль, образование, операция, свадьба. Объединим недвижимость и жилье в одну категорию. Их можно будет раздробить, если возникнет необходимость в более детальном анализе. Создадим столбцы purpose_type и purpose_id

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Лемматизация прошла успешно, удалось выделить основные цели получения кредита.
 
</div>

### Категоризация данных

In [6]:
def children_group(children):
    if children > 0:
        return 1
    else:
        return 0
    

def income_group(income):
    if income < 100000:
        return 'до 100 тыс'
    elif income < 150000:
        return '100 - 150 тыс'
    elif income <= 200000:
        return '150 - 200 тыс'
    else:
        return 'больше 200 тыс'
def df_grouped_category(df_column):
    """
    Формирует таблицу для определенного столбца, сгруппированного по его значениям, 
    с подсчетом:
    общего количества заемщиков в каждой категории ('count'), 
    количества невернувших вовремя ('sum') и
    доли тех, кто не вернул в срок, от общего количества ('mean')
    """
    df_grouped = df.groupby(df_column)['debt'].agg({'count', 'sum', 'mean'})
    df_grouped = df_grouped.sort_values(by='mean', ascending=False)
    return df_grouped

df['income_group'] = df['total_income'].apply(income_group)
df['children_group'] = df['children'].apply(children_group)
print(df.head(10))


   children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   среднее             1   
3         3           4124         32   среднее             1   
4         0              0         53   среднее             1   
5         0            926         27    высшее             0   
6         0           2879         43    высшее             0   
7         0            152         50   среднее             1   
8         2           6929         35    высшее             0   
9         0           2188         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M

**Вывод**

Категоризация проводилась с учётом заданных вопросов:

Есть ли зависимость между наличием детей и возвратом кредита в срок?

Есть ли зависимость между семейным положением и возвратом кредита в срок?

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Как разные цели кредита влияют на его возврат в срок?

Дя ответа на вопросы была проведена разбивка на несколько уровней дохода и упрощение данных по количеству детей до "есть дети / нет детей".

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Хорошо, у тебя получилось разбить клиентов на группы по уровню дохода. А почему ты выбрала именно такие границы?
    
Как вариант, можно воспользоваться статистическими характертисиками и разбить по квантилям. Подробнее про этот способ можешь прочитать вот <a href='https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.quantile.html'>здесь.</a>
 
</div>

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
print(df_grouped_category('children_group'))

                    mean   sum  count
children_group                       
1               0.092082   678   7363
0               0.075438  1063  14091


**Вывод**

Построим таблицу, отображающую, возвращают ли заёмщики кредит в срок в зависимости от наличия детей. Столбец 'children_group' отображает наличие детей (1 - есть, 0 - нет).
Столбец 'count' - сколько всего заёмщиков определенного типа, 'sum' - сколько не вернули кредит в срок, 'mean' - доля невернувших кредит в срок. Заёмщики без детей в 7.5% случаев не возвращают кредит в срок, а те, у кого есть дети, - в 9.2% случаев.

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Молодец, что автоматизировала процесс и написала функцию. Так код выглядит более структурированным.
    
Верно,заемщики без детей самые ответственные)
 
</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
print(df_grouped_category('family_status'))
df_grouped = df.groupby(['family_status', 'children_group'])['debt'].agg({'count', 'sum', 'mean'})
print(df_grouped)

                           mean  sum  count
family_status                              
Не женат / не замужем  0.097509  274   2810
гражданский брак       0.093471  388   4151
женат / замужем        0.075452  931  12339
в разводе              0.071130   85   1195
вдовец / вдова         0.065693   63    959
                                          mean  sum  count
family_status         children_group                      
Не женат / не замужем 0               0.092838  210   2262
                      1               0.116788   64    548
в разводе             0               0.070153   55    784
                      1               0.072993   30    411
вдовец / вдова        0               0.062574   53    847
                      1               0.089286   10    112
гражданский брак      0               0.083883  229   2730
                      1               0.111893  159   1421
женат / замужем       0               0.069095  516   7468
                      1               0.085

**Вывод**

Построим таблицу, отображающую, возвращают ли заёмщики кредит в срок в зависимости от семейного статуса. Наименьшее значение 'mean' у заёмщиков в статусе 'вдовец / вдова', следом 'в разводе', в середине - 'женат / замужем' и больше всего недоверия вызывают 'Не женат / не замужем' и 'гражданский брак', причём отрыв от остальных трёх категорий сильнее, чем разброс в трёх наиболее заслуживающих доверие категориях. Процент невозврата кредита в срок варьируется от 6.6% до 9.8%. 

При этом, категории 'Не женат / не замужем' и 'гражданский брак' при наличии детей ещё хуже соблюдают сроки возврата кредита. Лучше всех возвращают в срок люди в категории 'вдовец / вдова' без детей.

<div class="alert alert-success">
<h2> Комментарий от ревьюера <a class="tocSkip"></a></h2>
Посмотрев на сводную таблицу, можно увидеть, что доля просроченых кредитов больше у людей, которые находятся в гражданском браке или не женаты. Чаще всего возращают кредиты вдовцы и вдовы.
</div>


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
print(df_grouped_category('income_group'))

                    mean  sum  count
income_group                        
100 - 150 тыс   0.087151  624   7160
150 - 200 тыс   0.085013  405   4764
до 100 тыс      0.079319  354   4463
больше 200 тыс  0.070653  358   5067


**Вывод**

Хуже всего с возвращением кредита в срок у людей с доходом 100-200 тыс (8.5 - 8.7% не возвращает в срок). Лучше всего у 'крайних' групп: тех, у кого доход больше 200 тыс, и тех, кто зарабатывает меньше 100 тыс (7.0 - 7.9%).
Разброс менее ярко выражен, чем когда сравнивали по семейному положению. Как вариант, можно сделать более частую разбивку по доходам.

<div class="alert alert-success">
<h2> Комментарий от ревьюера <a class="tocSkip"></a></h2>
Молодец, верный вывод!
</div>


- Как разные цели кредита влияют на его возврат в срок?

In [10]:
print(df_grouped_category('purpose_type'))

                  mean  sum  count
purpose_type                      
автомобиль    0.093590  403   4306
образование   0.092200  370   4013
свадьба       0.080034  186   2324
недвижимость  0.072334  782  10811


**Вывод**

Кредиты на недвжимость и свадьбу не выплачиваются в срок в 7.2 - 8.0% случаев. На образование и автомобиль - в 9.2 и 9.4 случаев. Цель "недвижимость" - самая многочисленная. Возможно, полезно сделать дополнительную разбивку на жилую и коммерческую.

## Шаг 4. Общий вывод

Выявлены зависимости возврата кредита в срок от различных условий: наличия детей, семейного положения, уровня дохода и целей. Наиболее сильный разброс по доле невозврата в срок - по семейному положению. Большего доверия заслуживают вдовцы (особенно без детей) и разведенные, меньше - неженатые / незамужние или живущие в гражданском браке (особенно с детьми).

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>

Поздравляю с первым успешным  проектом!

- Ты хорошо делаешь выводы и анализируешь результаты.
    
- Код написан аккуратно и качественно, видно, что есть какой-то определенный бэкграунд.

- Соблюдена структура проекта, было приятно проверять:)
    
- Все написано четко и по делу

 


Желаю дальнейших успехов!


</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [ ]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [ ]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [ ]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [ ]  данные категоризированы;
- [ ]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.